In [1]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import warnings
warnings.filterwarnings("ignore")

f=open('./chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /Users/patshih/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/patshih/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
print(sent_tokens[:2])
print(word_tokens[:2])

['a chatbot is a software application used to conduct an on-line chat conversation via text or text-to-speech, in lieu of providing direct contact with a live human agent.', '[1] designed to convincingly simulate the way a human would behave as a conversational partner, chatbot systems typically require continuous tuning and testing, and many in production remain unable to adequately converse or pass the industry standard turing test.']
['a', 'chatbot']


In [3]:
#WordNet is a semantically-oriented dictionary of English included in NLTK.
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# remove punctuation, tokenize, and lemmatize in one call
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [4]:
# Default greeting messages
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence): 
    for word in sentence.split():
        # If user said hello, greet back
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(user_response):
    robo_response=''
    
    # add user input to sentence tokens
    sent_tokens.append(user_response)
    # convert sentence tokens to TF-IDF feature matrix [document, word][idf]
    tfidfvec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = tfidfvec.fit_transform(sent_tokens)
    # calculate cosine similarity between user input to each TF-IDF document (sentence) 
    vals = cosine_similarity(tfidf[-1], tfidf)
    # sort cosine similarity values in ascending order, 
    # then select index of highest cosine similarity value, excluding user input
    idx=vals.argsort()[0][-2]
    # convert from 2D to 1D array
    flat = vals.flatten()
    # sort cosine similarity values in ascending order
    flat.sort()
    # selecting highest cosine similarity, exclusing user input
    similarity = flat[-2]
    if(similarity==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [6]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type 'bye'!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")


ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type 'bye'!


 bye


ROBO: Bye! take care..
